In [ ]:
import pandas as pd
import requests
import numpy as np
from scipy.stats import poisson

requisicao = requests.get("https://pt.wikipedia.org/wiki/Campeonato_Brasileiro_de_Futebol_de_2025_-_S%C3%A9rie_A")

tabelas = pd.read_html(requisicao.text)
tabela_classificacao = tabelas[7]
tabela_jogos = tabelas[8]

nomes_times = list(tabela_jogos["Mandante \ Visitante"])

tabela_jogos_ajustada = tabela_jogos.set_index("Mandante \ Visitante")  # Índice correto
tabela_jogos_ajustada = tabela_jogos_ajustada.unstack().reset_index()
# Corrigindo os nomes das colunas corretamente
tabela_jogos_ajustada = tabela_jogos_ajustada.rename(columns={"Mandante \ Visitante": "casa", "Casa \ Fora": "fora", 0: "resultado"})
tabela_jogos_ajustada = tabela_jogos_ajustada.rename(columns={"level_0": "fora"})

tabela_jogos_ajustada["resultado"] = tabela_jogos_ajustada["resultado"].fillna("a jogar")
#tabela_jogos_ajustada


tabela_jogos_realizados = tabela_jogos_ajustada[tabela_jogos_ajustada["resultado"].str.contains("–")]
#print(tabela_jogos_realizados)

tabela_jogos_faltantes = tabela_jogos_ajustada
#print(tabela_jogos_faltantes)
tabela_jogos_faltantes = tabela_jogos_faltantes.drop(columns=["resultado"])
#display(tabela_jogos_faltantes)

# Separar os gols feitos pelo time da casa e os gols feitos pelo time visitante
tabela_jogos_realizados["gols_casa"] = tabela_jogos_realizados["resultado"].str.split("–").str[0].astype(int)
tabela_jogos_realizados["gols_fora"] = tabela_jogos_realizados["resultado"].str.split("–").str[1].astype(int)

# Calcular as médias de gols feitos e sofridos pelos times jogando em casa
media_gols_casa = tabela_jogos_realizados.groupby("casa").mean(numeric_only=True)

# Renomear as colunas para representar gols feitos e sofridos
media_gols_casa = media_gols_casa.rename(columns={"gols_casa": "gols_feitos_casa", "gols_fora": "gols_sofridos_casa"})

# Exibir o resultado
#print(media_gols_casa)
#display(media_gols_casa)

# Converte as colunas de gols para numérico, caso necessário
tabela_jogos_realizados["gols_casa"] = pd.to_numeric(tabela_jogos_realizados["gols_casa"], errors="coerce")
tabela_jogos_realizados["gols_fora"] = pd.to_numeric(tabela_jogos_realizados["gols_fora"], errors="coerce")

# media gols feitos dentro de casa
# media gols feitos fora de casa
# media gols sofridos dentro de casa
# media gols sofridos fora de casa

media_gols_casa = tabela_jogos_realizados.groupby("casa").mean(numeric_only=True)
#print(media_gols_casa)
media_gols_casa = media_gols_casa.rename(columns={"gols_casa": "gols_feitos_casa", "gols_fora": "gols_sofridos_casa"})
#display(media_gols_casa)
media_gols_fora = tabela_jogos_realizados.groupby("fora").mean(numeric_only=True)
media_gols_fora = media_gols_fora.rename(columns={"gols_casa": "gols_sofridos_fora", "gols_fora": "gols_feitos_fora"})
#display(media_gols_fora)

tabela_estatisticas_casa = media_gols_casa
tabela_estatisticas_fora =media_gols_fora

# Combinar as tabelas casa e fora, preenchendo os NaNs
tabela_estatisticas_unificada = tabela_estatisticas_casa.combine_first(tabela_estatisticas_fora)

# Definir o nome do índice como 'time'
tabela_estatisticas_unificada.index.name = 'time'

tabela_estatisticas_unificada = tabela_estatisticas_unificada.fillna(0)
#display(tabela_estatisticas_unificada)

# Função para calcular as probabilidades com base na distribuição de Poisson
def calcular_probabilidades(media_mandante, media_visitante, max_gols=10):
    prob_mandante = 0
    prob_empate = 0
    prob_visitante = 0

    for gols_casa in range(max_gols + 1):
        for gols_fora in range(max_gols + 1):
            p = poisson.pmf(gols_casa, media_mandante) * poisson.pmf(gols_fora, media_visitante)
            if gols_casa > gols_fora:
                prob_mandante += p
            elif gols_casa == gols_fora:
                prob_empate += p
            else:
                prob_visitante += p

    return prob_mandante, prob_empate, prob_visitante

# Lista para armazenar as probabilidades de cada jogo
probabilidades = []

# Loop sobre as linhas da tabela
for index, row in tabela_jogos_faltantes.iterrows():
    media_mandante = row.get('media_gols_mandante_casa', np.nan)
    media_visitante = row.get('media_gols_visitante_fora', np.nan)

    if pd.notnull(media_mandante) and pd.notnull(media_visitante):
        prob = calcular_probabilidades(media_mandante, media_visitante)
    else:
        prob = (np.nan, np.nan, np.nan)

    probabilidades.append(prob)

# Criar um DataFrame com as probabilidades
def calcular_probabilidades(odd_casa, odd_empate, odd_visitante):
    # Calcula as probabilidades implícitas
    prob_casa = 1 / odd_casa
    prob_empate = 1 / odd_empate
    prob_visitante = 1 / odd_visitante

    # Normaliza as probabilidades para garantir que somem 100%
    soma_probabilidades = prob_casa + prob_empate + prob_visitante
    prob_casa_normalizada = (prob_casa / soma_probabilidades) * 100
    prob_empate_normalizada = (prob_empate / soma_probabilidades) * 100
    prob_visitante_normalizada = (prob_visitante / soma_probabilidades) * 100

    return {
        "Vitória Casa": prob_casa_normalizada,
        "Empate": prob_empate_normalizada,
        "Vitória Visitante": prob_visitante_normalizada
    }

# Exemplo de uso
odds = {"casa": 2.62, "empate": 3.45, "visitante": 6.6}
probabilidades = calcular_probabilidades(odds["casa"], odds["empate"], odds["visitante"])
df_probabilidades = pd.DataFrame(probabilidades, columns=['prob_casa', 'prob_empate', 'prob_fora'])

# Concatenar com a tabela original
tabela_jogos_faltantes = pd.concat([tabela_jogos_faltantes.reset_index(drop=True), df_probabilidades], axis=1)


from matplotlib.pylab import poisson


def calcular_pontuacao_esperada(linha):
    time_casa = [linha["casa"]]  # Acessa o valor da coluna 'casa' da linha
    time_fora = [linha["fora"]]  # Acessa o valor da coluna 'fora' da linha

    # Média de gols do time casa jogando em casa contra o time fora jogando fora
    lambda_casa = (tabela_estatisticas_unificada.loc[time_casa, "gols_feitos_casa"]
                * tabela_estatisticas_unificada.loc[time_fora, "gols_sofridos_fora"])
    # Média de gols do time fora jogando fora contra o time casa jogando em casa
    lambda_fora = (tabela_estatisticas_unificada.loc[time_fora, "gols_feitos_fora"]
                * tabela_estatisticas_unificada.loc[time_casa, "gols_sofridos_casa"])

    # Inicializando as probabilidades
    pv_casa = 0
    p_empate = 0
    pv_fora = 0

    for gols_casa in range(8):  # Até 8 gols para casa
        for gols_fora in range(8):  # Até 8 gols para fora
            probabilidade_resultado = poisson.pmf(gols_casa, lambda_casa) * poisson.pmf(gols_fora, lambda_fora)
            if gols_casa == gols_fora:
                p_empate += probabilidade_resultado
            elif gols_casa > gols_fora:
                pv_casa += probabilidade_resultado
            else:
                pv_fora += probabilidade_resultado

    # Cálculo de pontuações esperadas
    ve_casa = pv_casa * 3 + p_empate
    ve_fora = pv_fora * 3 + p_empate

    # Adiciona os valores calculados à linha
    linha["pontos_casa"] = ve_casa
    linha["pontos_fora"] = ve_fora
    return linha

tabela_estatisticas = tabela_estatisticas_unificada
# Transformar o índice 'time' em uma coluna
tabela_estatisticas = tabela_estatisticas.reset_index()
#tabela_estatisticas


from scipy.stats import poisson
import time
# como a partir das medias calcular a probabilidade do time ganhar/perder/empate

def calcular_pontuacao_esperada(linha):
    time_casa = linha["casa"]
    time_fora = linha["fora"]

    # qual a média de gols que o Cruzeiro, jogando em casa, faz contra o Vasco, jogando fora de casa
    lambda_casa = (tabela_estatisticas.loc[tabela_estatisticas["time"]==time_casa, "gols_feitos_casa"].iloc[0] 
                * tabela_estatisticas.loc[tabela_estatisticas["time"]==time_fora, "gols_sofridos_fora"].iloc[0])
    # média de gols cruzeiro dentro de casa * média de gols sofridos pelo vasco fora de casa


    # média de gols que o vasco faz fora de casa * media de gols que o Cruzeiro sofre dentro de casa
    lambda_fora =  (tabela_estatisticas.loc[tabela_estatisticas["time"]==time_fora, "gols_feitos_fora"].iloc[0] 
                    * tabela_estatisticas.loc[tabela_estatisticas["time"]==time_casa, "gols_sofridos_casa"].iloc[0])



    pv_casa = 0
    p_empate = 0
    pv_fora = 0

    for gols_casa in range(8):
        for gols_fora in range(8):
            probabilidade_resultado = poisson.pmf(gols_casa, lambda_casa) * poisson.pmf(gols_fora, lambda_fora)
            if gols_casa == gols_fora:
                p_empate += probabilidade_resultado
            elif gols_casa > gols_fora:
                pv_casa += probabilidade_resultado
            elif gols_casa < gols_fora:
                pv_fora += probabilidade_resultado

     #print(pv_casa)
     #print(p_empate)
     #print(pv_fora)

    ve_casa = pv_casa * 3 + p_empate
    ve_fora = pv_fora * 3 + p_empate
    linha["pontos_casa"] = ve_casa
    linha["pontos_fora"] = ve_fora
    return linha

# como a partir da probabilidade você calcula a pontuação que o time vai ter
# ValorEsperado - pontuação do vasco

# Vasco x Cruzeiro

# p_vasco = 50% * 3 + 10% * 1
# p_cruzeiro = 40% * 3 + 10% * 1

# 50% * 3 = 1.8
# 10%
# 40% * 3 = 1.2

# Limpar a coluna 'Pts' removendo quaisquer caracteres não numéricos
tabela_classificacao["Pts"] = tabela_classificacao["Pts"].fillna("").astype(str)
#tabela_classificacao["Pts"] = (tabela_classificacao["Pts"].str.extract('(\d+)').astype(int))
# Verificar a tabela ajustada
#display(tabela_classificacao)

def ajustar_nome_time(linha):
    for nome in nomes_times:
        if nome in linha["Equipevde"]:
            return nome

tabela_classificacao["time"] = tabela_classificacao.apply(ajustar_nome_time, axis=1)

# Corrigir a coluna "Pts"
tabela_classificacao["Pts"] = pd.to_numeric(tabela_classificacao["Pts"], errors='coerce').fillna(0).astype(int)

# Criar a tabela atualizada
tabela_classificacao_atualizada = tabela_classificacao[["time", "Pts"]]

#display(tabela_classificacao_atualizada)

# Função para calcular as probabilidades de vitória, empate e derrota
def calcular_probabilidades(time_casa, time_fora, tabela_estatisticas):
    # Verificar se os dados do time existem na tabela
    gols_feitos_casa = tabela_estatisticas.loc[tabela_estatisticas["time"] == time_casa, "gols_feitos_casa"]
    gols_sofridos_fora = tabela_estatisticas.loc[tabela_estatisticas["time"] == time_fora, "gols_sofridos_fora"]
    gols_feitos_fora = tabela_estatisticas.loc[tabela_estatisticas["time"] == time_fora, "gols_feitos_fora"]
    gols_sofridos_casa = tabela_estatisticas.loc[tabela_estatisticas["time"] == time_casa, "gols_sofridos_casa"]

    # Garantir que os valores não sejam vazios
    if gols_feitos_casa.empty or gols_sofridos_fora.empty or gols_feitos_fora.empty or gols_sofridos_casa.empty:
        #print(f"Erro: Dados ausentes para os times {time_casa} ou {time_fora}")
        return 0, 0, 0  # Retornar probabilidades nulas

    lambda_casa = gols_feitos_casa.iloc[0] * gols_sofridos_fora.iloc[0]
    lambda_fora = gols_feitos_fora.iloc[0] * gols_sofridos_casa.iloc[0]

    pv_casa = 0
    p_empate = 0
    pv_fora = 0

    for gols_casa in range(8):  # Limite de 8 gols
        for gols_fora in range(8):
            probabilidade_resultado = poisson.pmf(gols_casa, lambda_casa) * poisson.pmf(gols_fora, lambda_fora)
            if gols_casa > gols_fora:
                pv_casa += probabilidade_resultado
            elif gols_casa == gols_fora:
                p_empate += probabilidade_resultado
            else:
                pv_fora += probabilidade_resultado

    return pv_casa, p_empate, pv_fora

# Adicionar a análise como nova coluna
def adicionar_analise(tabela_jogos_faltantes, tabela_estatisticas):
    resultados = []

    for _, linha in tabela_jogos_faltantes.iterrows():
        time_casa = linha["casa"]
        time_fora = linha["fora"]

        pv_casa, p_empate, pv_fora = calcular_probabilidades(time_casa, time_fora, tabela_estatisticas)

        if pv_casa > p_empate and pv_casa > pv_fora:
            resultado = "Vitória casa"
        elif pv_fora > pv_casa and pv_fora > p_empate:
            resultado = "Vitória fora"
        else:
            resultado = "Empate"

        resultados.append(resultado)

    tabela_jogos_faltantes["resultado_previsto"] = resultados
    return tabela_jogos_faltantes

# Exemplo de uso
# Certifique-se de que `tabela_estatisticas` contém as colunas:
# "time", "gols_feitos_casa", "gols_sofridos_casa", "gols_feitos_fora", "gols_sofridos_fora"

# Aplicar a função
tabela_jogos_faltantes = adicionar_analise(tabela_jogos_faltantes, tabela_estatisticas)

# Exibir a tabela atualizada
#display(tabela_jogos_faltantes)

# Excluir a coluna 'time'
tabela_classificacao_sem_time = tabela_classificacao.drop(columns=['time'])

# Redefinir o índice para remover o índice original
tabela_classificacao_sem_time.reset_index(drop=True, inplace=True)

# Agora o DataFrame não tem a coluna 'time' nem o índice original
#display(tabela_classificacao_sem_time)

# Exibir o DataFrame sem o índice
#print(tabela_classificacao_sem_time.to_string(index=False))

# Ou, ao salvar em um arquivo CSV:
tabela_classificacao_sem_time.to_csv('arquivo.csv', index=False)

colunas_remover = ['media_gols_casa', 'gols_feitos_fora']
colunas_existentes = [col for col in colunas_remover if col in media_gols_fora.columns]
apelidos_times = media_gols_fora.drop(columns=colunas_existentes)
#apelidos_times

# Análise das médias de gols feitos e sofridos Do Maior para o Menor
analise_exploratoria_gols_casa = media_gols_casa[media_gols_casa['gols_feitos_casa'] > 2.5 ].sort_values(by='gols_feitos_casa', ascending=False)
analise_exploratoria_gols_sofridos = media_gols_fora[media_gols_fora['gols_sofridos_fora'] > 2.5 ].sort_values(by='gols_sofridos_fora', ascending=False)

# Exibir os dados ordenados
#display(analise_exploratoria_gols_casa)
#display(analise_exploratoria_gols_sofridos)

# Vamos analisar os times que vão jogar fora de casa para trabalhar Lay (Metodo Novo)
analise_exploratoria_gols_casa = media_gols_casa[media_gols_casa['gols_feitos_casa'] > 0]
analise_exploratoria_gols_sofridos = media_gols_fora[media_gols_fora['gols_sofridos_fora'] > 0 ]

tabela_classificacao.to_csv('tabela_classificacao_atualizada.csv', index=False)

# A partir dessse momento podemos fazer as analises pré Live
# Após da definição dos jogos que iremos trabalhar, vamoos fazer uma pré analise 
times_escolhidos = ['Vasco da Gama','Red Bull Bragantino']
times_desejados = times_escolhidos
# Agora vou filtrar o time que vai jogar contra esse time da analise
filtro1 = analise_exploratoria_gols_casa.loc[analise_exploratoria_gols_casa.index == 'Vasco da Gama']
filtro2 = analise_exploratoria_gols_sofridos.loc[analise_exploratoria_gols_sofridos.index == 'RBB']
# Filtrando um confronto em especifico
jogos_filtrados_bot = tabela_jogos_faltantes[   (tabela_jogos_faltantes['casa'] == 'Vasco da Gama')
                                             & (tabela_jogos_faltantes['fora'] == 'RBB')
]

# Corrigindo a coluna V que está como bool e deve ser int
tabela_classificacao['V'] = pd.to_numeric(tabela_classificacao['V'], errors='coerce')
# Calcula o aproveitamento de vitórias e pontos
tabela_classificacao['Aproveitamento_Vitorias (%)'] = (tabela_classificacao['V'] / tabela_classificacao['J']) * 100
tabela_classificacao['Aproveitamento_Pontos (%)'] = (tabela_classificacao['Pts'] / (tabela_classificacao['J'] * 3)) * 100

# Arredondamento
tabela_classificacao['Aproveitamento_Vitorias (%)'] = tabela_classificacao['Aproveitamento_Vitorias (%)'].round(2)
tabela_classificacao['Aproveitamento_Pontos (%)'] = tabela_classificacao['Aproveitamento_Pontos (%)'].round(2)




filtro = tabela_classificacao[tabela_classificacao['time'].isin(times_desejados)]

#display(filtro[['time','Pts','Aproveitamento_Pontos (%)']])

 
tabela_classificacao_sem_time = tabela_classificacao
filtro_times = tabela_classificacao_sem_time[tabela_classificacao_sem_time['Equipevde'].isin(times_escolhidos)]






<>:12: SyntaxWarning: invalid escape sequence '\ '
<>:14: SyntaxWarning: invalid escape sequence '\ '
<>:17: SyntaxWarning: invalid escape sequence '\ '
<>:17: SyntaxWarning: invalid escape sequence '\ '
<>:12: SyntaxWarning: invalid escape sequence '\ '
<>:14: SyntaxWarning: invalid escape sequence '\ '
<>:17: SyntaxWarning: invalid escape sequence '\ '
<>:17: SyntaxWarning: invalid escape sequence '\ '
C:\Users\Rosana Alves\AppData\Local\Temp\ipykernel_7032\2710249672.py:12: SyntaxWarning: invalid escape sequence '\ '
  nomes_times = list(tabela_jogos["Mandante \ Visitante"])
C:\Users\Rosana Alves\AppData\Local\Temp\ipykernel_7032\2710249672.py:14: SyntaxWarning: invalid escape sequence '\ '
  tabela_jogos_ajustada = tabela_jogos.set_index("Mandante \ Visitante")  # Índice correto
C:\Users\Rosana Alves\AppData\Local\Temp\ipykernel_7032\2710249672.py:17: SyntaxWarning: invalid escape sequence '\ '
  tabela_jogos_ajustada = tabela_jogos_ajustada.rename(columns={"Mandante \ Visitante": 